In [1]:
%load_ext autoreload
%autoreload 2

from api_tricount import *


In [83]:
%load_ext autoreload
%autoreload 2
from api_portfolio import build_data


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import streamlit as st
import gspread as gs 
import pandas as pd
import numpy as np
import yfinance as yf

import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)


gc = gs.service_account_from_dict(st.secrets['gcp_service_account'])
ss = gc.open_by_key(st.secrets['portfolio'].spreadsheet_key)
dicts = pd.DataFrame(ss.worksheet('Dict').get_all_records())
operation = pd.DataFrame(ss.worksheet('Operations').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
greenbull = pd.DataFrame(ss.worksheet('GREENBULL').get_all_records()).sort_values('Date').astype({'Date': 'datetime64[ns]'}).set_index('Date')
assets = dicts.set_index('Asset')
print(assets)

market = yf.download(' '.join([*assets['Forex'], *assets['Market'][:-1]]), start='2021-04-28')['Close']
market = pd.concat([market, greenbull], axis=1).ffill().bfill()
print(market.columns)


              Market Currency     Forex           Class  PriceFmt AmountFmt  \
Asset                                                                         
EUR         EUREUR=X           EUREUR=X         Deposit                       
€           EUREUR=X      EUR  EUREUR=X            Cash  {:.0f} €    {:.0f}   
$              EUR=X      EUR  EUREUR=X            Cash  {:.4f} €    {:.0f}   
USDT           EUR=X     EUR   EUREUR=X            Cash  {:.4f} €    {:.0f}   
ESE           ESE.PA        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
PUST                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
RS2K                        €  EUREUR=X        Equities  {:.2f} €    {:.0f}   
OBLI         OBLI.PA        €  EUREUR=X    Fixed Income  {:.2f} €    {:.0f}   
GOLD         GOLD.PA        €  EUREUR=X     Commodities  {:.2f} €    {:.0f}   
BTC          BTC-USD     USDT  EURUSD=X  Cryptocurrency  {:.0f} $    {:.4f}   
ETH                      USDT  EURUSD=X  Cryptocurre

In [22]:

df = pd.DataFrame(index=market.index, columns=pd.MultiIndex(levels=[[],[],[],[]], codes=[[],[],[],[]]))#, names=['portfolio', 'class', 'asset', 'metric']))
df.head()

for portfolio in np.unique(operation['Portfolio']):
    print(portfolio)
    dfp: pd.DataFrame = operation[operation['Portfolio'] == portfolio]

    print(f"\tAssets")
    for asset in np.unique(dfp['Asset']):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{classs}:\t{asset}")

        dfa: pd.DataFrame = dfp[dfp['Asset'] == asset]
        dfa = dfa.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)

        df[portfolio, classs, asset, 'Quantity'] = dfa['Quantity']
        df[portfolio, classs, asset, 'Operation'] = dfa['Operation']
        df[portfolio, classs, asset, 'Market'] = market[assets.loc[asset]['Market']]
        df[portfolio, classs, asset, 'Forex'] = market[assets.loc[asset]['Forex']]
    
    print(f"\tCurrency")
    for currency in np.unique(dfp['Currency']):
        classs = assets.loc[currency]['Class']
        print(f"\t\t{classs}:\t{currency}")

        dfc: pd.DataFrame = dfp[dfp['Currency'] == currency]
        dfc = dfc.groupby('Date').agg({'Quantity': 'sum', 'Operation': 'sum'}).reindex(df.index).fillna(0)
        
        if (portfolio, classs, currency, 'Quantity') not in df.columns:
            df[portfolio, classs, currency, 'Quantity'] = 0
        if (portfolio, classs, currency, 'Operation') not in df.columns:
            df[portfolio, classs, currency, 'Operation'] = 0
        if (portfolio, classs, currency, 'Market') not in df.columns:
            df[portfolio, classs, currency, 'Market'] = market[assets.loc[currency]['Market']]

        df[portfolio, classs, currency, 'Quantity'] -= dfc['Operation']
        df[portfolio, classs, currency, 'Operation'] -= dfc['Operation'] * df[portfolio, classs, currency, 'Market']


    print(f"\tBuild datatable ...")
    for asset in np.unique([*dfp['Asset'], *dfp['Currency']]):
        classs = assets.loc[asset]['Class']
        print(f"\t\t{asset}")

        toEUR = market[assets.loc[asset]['Forex']]
        df[portfolio, classs, asset, 'OperationEUR'] = df[portfolio, classs, asset, 'Operation'] / toEUR

        position, invested, investedEUR = pd.Series(dtype=float), pd.Series(dtype=float), pd.Series(dtype=float)
        for idx in df.index:
            position[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Quantity'])
            invested[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'Operation'])
            investedEUR[idx] = np.sum(df.loc[:idx][portfolio, classs, asset, 'OperationEUR'])

        df = pd.concat([df, pd.DataFrame({
                (portfolio, classs, asset, 'Position'): position,
                (portfolio, classs, asset, 'Invested'): invested,
                (portfolio, classs, asset, 'InvestedEUR'): investedEUR,
            })], axis=1)

        df[portfolio, classs, asset, 'PRU'] = df[portfolio, classs, asset, 'Invested'] / df[portfolio, classs, asset, 'Position']
        df[portfolio, classs, asset, 'Value'] = df[portfolio, classs, asset, 'Position'] * df[portfolio, classs, asset, 'Market']
        df[portfolio, classs, asset, 'PnL'] = df[portfolio, classs, asset, 'Value'] - df[portfolio, classs, asset, 'Invested']
            
        df[portfolio, classs, asset, 'ValueEUR'] = df[portfolio, classs, asset, 'Value'] / toEUR
        df[portfolio, classs, asset, 'PnLEUR'] = df[portfolio, classs, asset, 'PnL'] / toEUR
        df[portfolio, classs, asset, 'PnLEUR2'] = df[portfolio, classs, asset, 'ValueEUR'] - df[portfolio, classs, asset, 'InvestedEUR']

        


    print(f"\tConcate class data ...")
    for classs in np.unique(df[portfolio].columns.get_level_values(0)):
        print(f"\t\t{classs}")
        df[portfolio, classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'InvestedEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'ValueEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, classs, :, 'PnLEUR']].sum(axis=1)
        df[portfolio, classs, 'All', 'PnLEUR2'] = df[portfolio, classs, 'All', 'ValueEUR'] - df[portfolio, classs, 'All', 'InvestedEUR']

    # df[portfolio, 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'InvestedEUR']].sum(axis=1)
    # df[portfolio, 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'ValueEUR']].sum(axis=1)
    # df[portfolio, 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[portfolio, :, 'All', 'PnLEUR']].sum(axis=1)

    print(f"... OK")

for classs in np.unique(df.columns.get_level_values(1)):
    df['All', classs, 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'InvestedEUR']].sum(axis=1)
    df['All', classs, 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'ValueEUR']].sum(axis=1)
    df['All', classs, 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice[:, classs, 'All', 'PnLEUR']].sum(axis=1)
    df['All', classs, 'All', 'PnLEUR2'] = df['All', classs, 'All', 'ValueEUR'] - df['All', classs, 'All', 'InvestedEUR']

# df['All', 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'InvestedEUR']].sum(axis=1)
# df['All', 'All', 'All', 'ValueEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'ValueEUR']].sum(axis=1)
# df['All', 'All', 'All', 'PnLEUR'] = df.loc[:, pd.IndexSlice['All', :, 'All', 'PnLEUR']].sum(axis=1)


for column in df.columns:
    print(column)


DMA
	Assets
		Cash:	€
	Currency
		Deposit:	EUR
	Build datatable ...
		EUR
		€
	Concate class data ...
		Cash
		Deposit
... OK
ZEN
	Assets
		Cash:	$
		Cryptocurrency:	BTC
		Equities:	ESE
		Commodities:	GOLD
		Equities:	GREENBULL
		Fixed Income:	OBLI
		Cash:	USDT
		Cash:	€
	Currency
		Cash:	$
		Deposit:	EUR
		Cash:	USDT
		Cash:	€
	Build datatable ...
		$
		BTC
		ESE
		EUR
		GOLD
		GREENBULL
		OBLI
		USDT
		€
	Concate class data ...
		Cash
		Commodities
		Cryptocurrency
		Deposit
		Equities
		Fixed Income
... OK
('DMA', 'Cash', '€', 'Quantity')
('DMA', 'Cash', '€', 'Operation')
('DMA', 'Cash', '€', 'Market')
('DMA', 'Cash', '€', 'Forex')
('DMA', 'Deposit', 'EUR', 'Quantity')
('DMA', 'Deposit', 'EUR', 'Operation')
('DMA', 'Deposit', 'EUR', 'Market')
('DMA', 'Deposit', 'EUR', 'OperationEUR')
('DMA', 'Deposit', 'EUR', 'Position')
('DMA', 'Deposit', 'EUR', 'Invested')
('DMA', 'Deposit', 'EUR', 'InvestedEUR')
('DMA', 'Deposit', 'EUR', 'PRU')
('DMA', 'Deposit', 'EUR', 'Value')
('DMA', 'Deposit'

In [26]:
dates = ["2021-04-30","2021-07-28","2021-07-29","2021-07-30","2021-08-10","2021-10-01","2021-11-01","2021-12-01","2021-12-31","2022-02-01","2022-03-24","2022-04-01","2022-04-05","2022-04-13","2022-05-02","2022-04-19","2022-06-01","2022-07-01","2022-07-04","2022-07-05","2022-08-05","2022-08-25","2022-10-12","2023-02-10"]

# df['ZEN', 'All', 'All', 'InvestedEUR'] = df.loc[:, pd.IndexSlice['ZEN', :, 'All', 'InvestedEUR']].sum(axis=1)
df.loc[dates, pd.IndexSlice['ZEN', ['Equities'], :, ['PnLEUR', 'ValueEUR', 'InvestedEUR', 'PnLEUR2']]]

ZEN                                                      \
             Equities                                                       
                  ESE    GREENBULL          All          ESE    GREENBULL   
               PnLEUR       PnLEUR       PnLEUR     ValueEUR     ValueEUR   
2021-04-30   0.000000   -37.210900   -37.210900     0.000000   455.899902   
2021-07-28   0.000000    -2.451128    -2.451128     0.000000   503.337270   
2021-07-29   0.000000    -2.446405    -2.446405     0.000000   502.367228   
2021-07-30   0.000000    -0.341377    -0.341377     0.000000   994.361638   
2021-08-10   0.000000    -5.641282    -5.641282     0.000000  1002.156396   
2021-10-01   0.000000   -10.839368   -10.839368     0.000000  1508.019918   
2021-11-01   0.000000    64.778957    64.778957     0.000000  2083.523957   
2021-12-01   0.000000   -53.749728   -53.749728     0.000000  2534.886610   
2021-12-31   0.000000   -44.711589   -44.711589     0.000000  2986.627510   
2022-02-01   0.000000  -204.298511  -204.298511     0.000000  3347.015508   
2022-03-24   0.000000  -462.207739  -462.207739     0.000000  3161.169488   
2022-04-01   0.000000  -274.056351  -274.056351     0.000000  3806.375336   
2022-04-05   0.000000  -276.465709  -276.465709     0.000000  3839.838964   
2022-04-13   0.000000  -280.289111  -280.289111     0.000000  3892.942288   
2022-05-02   0.000000  -689.468883  -689.468883     0.000000  3598.700273   
2022-04-19   0.000000  -281.442197  -281.442197     0.000000  3908.957534   
2022-06-01   0.000000  -726.773207  -726.773207     0.000000  3957.362031   
2022-07-01   0.000000 -1038.630929 -1038.630929     0.000000  3760.142425   
2022-07-04   0.000000 -1042.983917 -1042.983917     0.000000  3775.901495   
2022-07-05   0.000000 -1043.060068 -1043.060068     0.000000  3776.177181   
2022-08-05   0.000000  -822.071720  -822.071720     0.000000  4084.351046   
2022-08-25   0.000000  -845.053588  -845.053588     0.000000  4198.533320   
2022-10-12   5.464462 -1293.194127 -1287.729665   991.864462  3890.221373   
2023-02-10  41.731514  -518.350254  -476.618741  1028.131514  4162.214946   

                                                                          \
                                                                           
                    All         ESE    GREENBULL          All        ESE   
               ValueEUR InvestedEUR  InvestedEUR  InvestedEUR    PnLEUR2   
2021-04-30   455.899902         0.0   493.110802   493.110802   0.000000   
2021-07-28   503.337270         0.0   493.110802   493.110802   0.000000   
2021-07-29   502.367228         0.0   493.110802   493.110802   0.000000   
2021-07-30   994.361638         0.0   984.996908   984.996908   0.000000   
2021-08-10  1002.156396         0.0   984.996908   984.996908   0.000000   
2021-10-01  1508.019918         0.0  1481.779667  1481.779667   0.000000   
2021-11-01  2083.523957         0.0  1979.327191  1979.327191   0.000000   
2021-12-01  2534.886610         0.0  2508.881164  2508.881164   0.000000   
2021-12-31  2986.627510         0.0  2950.381179  2950.381179   0.000000   
2022-02-01  3347.015508         0.0  3444.608681  3444.608681   0.000000   
2022-03-24  3161.169488         0.0  3444.608681  3444.608681   0.000000   
2022-04-01  3806.375336         0.0  3923.278180  3923.278180   0.000000   
2022-04-05  3839.838964         0.0  3923.278180  3923.278180   0.000000   
2022-04-13  3892.942288         0.0  3923.278180  3923.278180   0.000000   
2022-05-02  3598.700273         0.0  3923.278180  3923.278180   0.000000   
2022-04-19  3908.957534         0.0  3923.278180  3923.278180   0.000000   
2022-06-01  3957.362031         0.0  4398.399296  4398.399296   0.000000   
2022-07-01  3760.142425         0.0  4398.399296  4398.399296   0.000000   
2022-07-04  3775.901495         0.0  4398.399296  4398.399296   0.000000   
2022-07-05  3776.177181         0.0  4398.399296  4398.399296   0.000000   
2022-08-05  4084.351046         0.0  

In [6]:
s = df.iloc[-1]
s.loc[pd.IndexSlice['ZEN', ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum()

11627.114336033424

In [19]:
rows = {}
for ptf in ['ZEN']:
    rows[ptf] = {
        'Value': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum(),
        'Invested': s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum(),
        'Cash': s.loc[pd.IndexSlice[ptf, 'Cash', 'All', 'ValueEUR']],
        'PnL': s.loc[pd.IndexSlice[ptf, ['Cash', 'Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'PnLEUR']].sum(),
        'Deposited': -s.loc[pd.IndexSlice[ptf, 'Deposit', 'All', 'InvestedEUR']],
    }
df1 = pd.DataFrame(rows).transpose()

print("deposited  : invested+cash(invested=value-pnl)        :", df1['Deposited'].values[0] , "\t:", df1['Invested'].values[0] + df1['Cash'].values[0] - s.loc[pd.IndexSlice[ptf, 'Cash', 'All', 'PnLEUR']])
print("pnl        : value-invested-cash(invested=value-pnl)  :", df1['PnL'].values[0]       , "\t:", df1['Value'].values[0] - df1['Invested'].values[0] )#- (df1['Cash'].values[0]))# - s.loc[pd.IndexSlice[ptf, 'Cash', 'All', 'PnLEUR']]))
print("pnl        : value-deposited                          :", df1['PnL'].values[0]       , "\t:", df1['Value'].values[0] - df1['Deposited'].values[0])

print(s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'ValueEUR']].sum())
print(s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'InvestedEUR']].sum())
print(s.loc[pd.IndexSlice[ptf, ['Commodities', 'Cryptocurrency', 'Equities', 'Fixed Income'], 'All', 'PnLEUR']].sum())


df1

deposited  : invested+cash(invested=value-pnl)        : 12000.0 	: 12000.000000024984
pnl        : value-invested-cash(invested=value-pnl)  : -664.4497409469874 	: 1954.4247994703164
pnl        : value-deposited                          : -664.4497409469874 	: -372.8856639665755
9237.718623612483
9672.689536563108
-726.5349899060518


,Cash,Deposited,Invested,PnL,Value
ZEN,2389.395712,12000.0,9672.689537,-664.449741,11627.114336
